In [1]:
import numpy as np
from osgeo import gdal
import pandas as pd
import matplotlib.pyplot as plt



# Load the two raster images
image1_path = '/home/jovyan/Dev/deafrica-sandbox-notebooks/Use_cases/Vegetated_wetlands_mapping/results/prediction_1995.tif'
image2_path = '/home/jovyan/Dev/deafrica-sandbox-notebooks/Use_cases/Vegetated_wetlands_mapping/results/prediction_2022.tif'

# Open the raster images
image1 = gdal.Open(image1_path)
image2 = gdal.Open(image2_path)

# Get the image size
image1_cols = image1.RasterXSize
image1_rows = image1.RasterYSize
image2_cols = image2.RasterXSize
image2_rows = image2.RasterYSize

In [2]:
# Check if the images have the same size
if image1_cols != image2_cols or image1_rows != image2_rows:
    raise ValueError("Image sizes do not match.")

In [3]:
# Get the geotransform to obtain pixel size and area
geotransform = image1.GetGeoTransform()
pixel_width = geotransform[1]
pixel_height = geotransform[5]
pixel_area_ha = abs(pixel_width * pixel_height / 10000)  # Convert to hectares

In [4]:
# Read the image data into numpy arrays
image1_data = image1.ReadAsArray()
image2_data = image2.ReadAsArray()

In [ ]:
# Create a dictionary to store change information
class_names = {
    1: 'Cropland',
    2: 'Dense Forest',
    3: 'Open Forest',
    4: 'Open Grassland',
    5: 'Open Water',
    6: 'Otherland',
    7: 'Settlements',
    8: 'Vegetated Wetland',
    9: 'Wooded Grassland'
}

change_info = {}

# Iterate through each pixel and detect changes
for row in range(image1_rows):
    for col in range(image1_cols):
        prev_class = image1_data[row, col]
        new_class = image2_data[row, col]
        if prev_class != new_class and new_class in class_names:
            prev_class_name = class_names.get(prev_class, f"Class {prev_class}")
            new_class_name = class_names.get(new_class, f"Class {new_class}")
            change_key = f"{prev_class_name} to {new_class_name}"
            if change_key not in change_info:
                change_info[change_key] = 1
            else:
                change_info[change_key] += 1

# Convert the change information from pixels to hectares
change_info_ha = {change: count * pixel_area_ha for change, count in change_info.items()}

# Prepare data for the chart
changes = list(change_info_ha.keys())
areas = list(change_info_ha.values())

# Create the bar chart
plt.bar(changes, areas)
plt.xlabel('Change')
plt.ylabel('Area (hectares)')
plt.title('Change Information in Hectares')
plt.xticks(rotation=90)

# Display the chart
#plt.show()

In [ ]:
# Convert the change information from pixels to hectares
change_info_ha = {change: count * pixel_area_ha for change, count in change_info.items()}

# Print the change information in hectares
for change, area in change_info_ha.items():
    print(f"{area:.2f}ha {change}")

In [ ]:
data = pd.DataFrame({
    'Change': [f"{change}" for change in change_info_ha.keys()],
    'Area (hectares)': [area for area in change_info_ha.values()]
})

# Save data to Excel file
excel_file_path = 'change_info_ha.xlsx'  # Replace with your desired file path
data.to_excel(excel_file_path, index=False)


In [ ]:
import matplotlib.patches as mpatches

In [ ]:
# Create a dictionary to store change information
class_names = {
    1: 'Cropland',
    2: 'Dense Forest',
    3: 'Open Forest',
    4: 'Open Grassland',
    5: 'Open Water',
    6: 'Otherland',
    7: 'Settlements',
    8: 'Vegetated Wetland',
    9: 'Wooded Grassland'
}


In [ ]:

# Create a binary mask to identify changes in classes 3, 4, and 8
change_mask_3 = np.where((image1_data != image2_data) & (image1_data == 8)  & (image2_data == 7), 1, 0)
change_mask_4 = np.where((image1_data != image2_data) & (image1_data == 8)& (image2_data == 6), 2, 0)
change_mask_8 = np.where((image1_data != image2_data) & (image1_data == 8)& (image2_data == 1), 3, 0)

# Set the colormap to white, red, green, and blue
cmap = plt.cm.colors.ListedColormap(['white','#000000'])

# Define the land cover classes
class_names = {
     #2: 'Forest Changed areas',
     #8: 'Vegetated Wetland'
     1: 'Landcover Changes to Settlements',
}


# Combine the masks to get the final change mask
change_mask = change_mask_3 + change_mask_4 + change_mask_8
# Plot the change map
plt.figure(figsize=(10, 10))
plt.imshow(change_mask, cmap=cmap)
plt.title('Landcover Changes to Settlements 2000-2022')
plt.xticks([])
plt.yticks([])

# Create legend handles and labels
legend_handles = []
legend_labels = []
for class_id, class_name in class_names.items():
    legend_handles.append(mpatches.Patch(color=cmap(class_id), label=class_name))
    legend_labels.append(class_name)

# Add legend
plt.legend(handles=legend_handles, labels=legend_labels, loc='lower right')

plt.savefig('Landcover Changes to Settlements 2000-2022.png',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import rasterio

In [ ]:
# Open the georeferenced image file
with rasterio.open(image1_path) as src:
    # Read the geotransform and CRS
    geotransform = src.transform
    crs = src.crs

In [ ]:
with rasterio.open(
    'Wetlands Depletion.tif',
    'w',
    driver='GTiff',
    width=change_mask.shape[1],
    height=change_mask.shape[0],
    count=1,
    dtype=np.uint8,
    transform=geotransform,
    crs=crs
) as dst:
    dst.write(change_mask, 1)